***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-0f91  GCE       3                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 26 10:35 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'irprojectaon' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def tokenize(text):
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]    
    return list_of_tokens

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  dic = {}
  returnLst = []
  for token in tokens:
    if token in dic:
      changeVal = dic[token]
      changeVal += 1
      dic[token] = changeVal
    else:
      if token in all_stopwords:
        continue
      dic.update({token: 1})
  for key in dic:
    returnLst.append((key,(id,dic[key])))
  return(returnLst)

def reduce_word_counts(unsorted_pl):
  newLst = {x: 0 for x, _ in unsorted_pl}
  for wiki_id, tf in unsorted_pl:
      newLst[wiki_id] += tf
  Output = list(map(tuple, newLst.items()))
  Output.sort(key=lambda a: a[0])
  return(Output)

def calculate_df(postings):
    newRdd = postings.map(lambda x: (x[0], len(x[1])))
    return newRdd


def partition_postings_and_write(postings):
  makeTup = postings.map(lambda x: (token2bucket_id(x[0]),(x[0],x[1])))
  checkRdd = makeTup.groupByKey().mapValues(list)
  finalRes = checkRdd.map(lambda y: InvertedIndex.write_a_posting_list(y, bucket_name, "text_big_index"))
  return finalRes

In [14]:
# DL = doc_text_pairs.map(lambda row: (row[1], len(tokenize(row[0])))).collectAsMap()
# with open('DL_BIG.pkl', 'wb') as handle:
#    pickle.dump(DL, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
# DL_D = "DL_BIG.pkl"
# DL_dict = f'gs://{bucket_name}/{DL_D}'
# !gsutil cp $DL_D $DL_dict

In [16]:
# print(len(DL.keys()))

In [17]:
# text index
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

postings_filtered = postings.filter(lambda x: len(x[1])>50)

w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [18]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'big_index')
# upload to gs
index_src = "big_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_text_big_index/{index_src}'
!gsutil cp $index_src $index_dst

NameError: name 'super_posting_locs' is not defined

In [ ]:
!gsutil ls -lh $index_text

In [ ]:
# PageRank

In [ ]:
# Put your `generate_graph` function here
def generate_graph(pages):
    edgeTest = pages.flatMap(lambda y: [(y.id, x.id) for x in y.anchor_text])
    edges = edgeTest.distinct()
    verticeTest = edges.flatMap(lambda x: [(x[0],),(x[1],)])
    vertices = verticeTest.distinct()
    return edges, vertices

In [ ]:
t_start = time()
#pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
pages_links = parquetFile.select("id","anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr_big', compression="gzip")
pr_time = time() - t_start
pr.show()

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*120

In [ ]:
# pr_key = 38153382
# pr_value = pr.filter(pr.id == pr_key).select("pagerank").collect()[0][0]
# print(f"The PageRank value for key {pr_key} is: {pr_value}")

In [ ]:
# # Get the 4000th value from the pr DataFrame
# value_4000th = pr.take(4000)[-1]

# # Show the 4000th value
# print(value_4000th)